# Compulsory 1 ML
MLP for cali housing market.

In [1]:
import tensorflow
import sys
from tensorflow import keras
from tensorflow.keras import layers
import numpy

# Sæt seeds så notebook producer samme vægte ved første træning, hver gang. Gør den reproducerbar.
numpy.random.seed(42)
tensorflow.random.set_seed(42)

# Fetch data and split

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Henter data
housing = fetch_california_housing()

# Separerer data til træning og validering
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Build model and compile

In [3]:
if "google.colab" in sys.modules:
  %pip install -q -U keras-tuner

import keras_tuner as kt

norm_layer = layers.Normalization()

def build_model(hp):
  n_hidden = hp.Int("n_hidden", min_value=2, max_value=10, default=2)
  n_neurons = hp.Int("n_neurons", min_value=10, max_value=200)
  learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
  optimizer = tensorflow.keras.optimizers.SGD(learning_rate=learning_rate)
  model = tensorflow.keras.Sequential()
  model.add(norm_layer)
  for _ in range(n_hidden):
      model.add(tensorflow.keras.layers.Dense(n_neurons, activation="relu"))
  model.add(tensorflow.keras.layers.Dense(1, activation="softmax"))
  model.compile(loss="mse", optimizer=optimizer,
                  metrics=["RootMeanSquaredError"])
  norm_layer.adapt(X_train)
  return model


# --- Normalization: adapt BEFORE building/fit ---
#norm_layer = layers.Normalization()

#model = keras.Sequential([
#    norm_layer,
#    layers.Dense(150, activation="relu"),
#    layers.Dense(150, activation="relu"),
#    layers.Dense(50, activation="relu"),
#    layers.Dense(50, activation="relu"),
#    layers.Dense(50, activation="relu"),
#    layers.Dense(1)
#])

#model.compile(loss="mse", optimizer="sgd", metrics=["RootMeanSquaredError"])

#norm_layer.adapt(X_train)

# Callbacks and Hypertuning

In [4]:
# TODO: Lav noget hypertuning her også.

# ModelCheckpoint.
checkpoint_cb = keras.callbacks.ModelCheckpoint("cali_houseing.keras", save_best_only=True)

# EarlyStopping (with rollback to the best model).
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# TODO: learning rate optimizer callback
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

# Train model

In [5]:
# TODO: brug hypertuning
random_search = kt.RandomSearch(
    build_model, objective="mse", max_trials=10, overwrite=True,
    directory="my_housing", project_name="housing_price", seed=42)

random_search.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler])

#history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler])

Trial 2 Complete [00h 00m 22s]

Best mse So Far: None
Total elapsed time: 00h 00m 50s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
6                 |7                 |n_hidden
22                |17                |n_neurons
0.00098872        |0.00065625        |learning_rate

Epoch 1/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - RootMeanSquaredError: 1.6069 - loss: 2.5824 - val_RootMeanSquaredError: 1.5447 - val_loss: 2.3861 - learning_rate: 9.8872e-04
Epoch 2/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 1.6069 - loss: 2.5824 - val_RootMeanSquaredError: 1.5447 - val_loss: 2.3861 - learning_rate: 9.8872e-04
Epoch 3/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 1.6069 - loss: 2.5824 - val_RootMeanSquaredError: 1.5447 - val_loss: 2.3861 - learning_rate: 9.8872e-04
Epoch 4/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 1.6069 - loss: 2.5824 - val_RootMeanSquaredError: 1.5447 - val_los

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/tuner_utils.py", line 145, in convert_to_metrics_dict
    best_value, _ = _get_best_value_and_best_epoch_from_history(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/tuner_utils.py", line 116, in _get_best_value_and

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/tuner_utils.py", line 145, in convert_to_metrics_dict
    best_value, _ = _get_best_value_and_best_epoch_from_history(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ke

In [ ]:
best_trail = random_search.oracle.get_best_trials(num_trials=1)[0]
best_trail.summary()

best_trail.metrics.get_last_value("val_root_mean_squared_error")

In [ ]:
# Optional: evaluate on test set
test_loss, test_rmse = model.evaluate(X_test, y_test, verbose=0)
print("Test RMSE:", test_rmse)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas

pandas.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()